In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.externals import joblib
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
import catboost as cb
import gc

%matplotlib inline

In [2]:
plt.rc('figure',dpi=100);

In [3]:
avisos = pd.read_csv("../../Data/fiuba_entrenamiento/gian/avisos.csv")
postulaciones = pd.read_csv("../../Data/fiuba_entrenamiento/gian/postulaciones.csv")
postulantes = pd.read_csv("../../Data/fiuba_entrenamiento/gian/postulantes.csv")

postulantes['sexo']=postulantes['sexo'].astype('category')
postulantes['Doctorado']=postulantes['Doctorado'].astype('category')
postulantes['Master']=postulantes['Master'].astype('category')
postulantes['Otro']=postulantes['Otro'].astype('category')
postulantes['Posgrado']=postulantes['Posgrado'].astype('category')
postulantes['Secundario']=postulantes['Secundario'].astype('category')
postulantes['Terciario/Técnico']=postulantes['Terciario/Técnico'].astype('category')
postulantes['Universitario']=postulantes['Universitario'].astype('category')

avisos['nombre_zona']=avisos['nombre_zona'].astype('category')
avisos['tipo_de_trabajo']=avisos['tipo_de_trabajo'].astype('category')
avisos['nivel_laboral']=avisos['nivel_laboral'].astype('category')
avisos['nombre_area']=avisos['nombre_area'].astype('category')

postulaciones['sepostulo']=1
postulaciones.drop(['fechapostulacion','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)

In [4]:
vistas = pd.read_csv("../../Data/fiuba_entrenamiento/gian/vistas.csv")
vistas.head()

,idAviso,timestamp,idpostulante,dia,mes,semana,diadelasemana,hora
0,1111780242,2018-02-23 18:38:13.187,YjVJQ6Z,23,2,8,Viernes,18
1,1112263876,2018-02-23 18:38:14.296,BmVpYoR,23,2,8,Viernes,18
2,1112327963,2018-02-23 18:38:14.329,wVkBzZd,23,2,8,Viernes,18
3,1112318643,2018-02-23 18:38:17.921,OqmP9pv,23,2,8,Viernes,18
4,1111903673,2018-02-23 18:38:18.973,DrpbXDP,23,2,8,Viernes,18


In [5]:
vistas.drop(['timestamp','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)
vistas['cantidad']=0
vistas2=vistas.groupby(['idAviso','idpostulante'],as_index=False).agg({'cantidad':'count'})
vistas2=vistas2.rename(columns={'idAviso': 'idaviso'})
vistas2.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,2
1,48375,RwVdKR,1
2,169730,1KjXB,2
3,169730,2AKzxa,2
4,169730,6LJ64,1


In [6]:
vistas=vistas.groupby(['idpostulante'],as_index=False).agg({'cantidad':'count'});
vistas=vistas.rename(columns={'idAviso': 'idaviso'})
vistas.head()

,idpostulante,cantidad
0,0002q,15
1,0005E,58
2,000R8,14
3,001XE,26
4,003k9,32


In [7]:
vistas2=pd.merge(vistas2,vistas,on='idpostulante',how='left')

In [8]:
vistas2.head()

,idaviso,idpostulante,cantidad_x,cantidad_y
0,18,BolNL,2,28
1,48375,RwVdKR,1,15
2,169730,1KjXB,2,99
3,169730,2AKzxa,2,2
4,169730,6LJ64,1,23


In [9]:
vistas2['cantidad']=vistas2['cantidad_x']/vistas2['cantidad_y']
vistas2.drop(['cantidad_x','cantidad_y'],axis=1,inplace=True)
vistas=vistas2
vistas2.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,0.071429
1,48375,RwVdKR,0.066667
2,169730,1KjXB,0.020202
3,169730,2AKzxa,1.000000
4,169730,6LJ64,0.043478


In [10]:
postulaciones2= pd.read_csv("../../Data/fiuba_entrenamiento/gian/no-postulaciones/7.csv")
postulaciones2=postulaciones2.append(pd.read_csv("../../Data/fiuba_entrenamiento/gian/no-postulaciones/8.csv").sample(len(postulaciones2)//2).reset_index(drop=True))
print(len(postulaciones2))
postulaciones2

9906801


,idaviso,idpostulante,sepostulo
0,1112389495,5NbOjG,0
1,1112024109,1QP841b,0
2,1112490884,96XQ6N9,0
3,1111474259,qe2QJj1,0
4,1112211292,KBzbBJm,0
5,1112328485,3NzZv26,0
6,1112474766,akDjWwE,0
7,1112312385,aZpBJW,0
8,1112285628,akDWj55,0
9,1112322843,GNd0KNN,0


In [11]:
postulaciones.head()

,idaviso,idpostulante,sepostulo
0,1112257047,NM5M,1
1,1111920714,NM5M,1
2,1112346945,NM5M,1
3,1112345547,NM5M,1
4,1112237522,5awk,1


In [12]:
postulaciones=postulaciones.append(postulaciones2, ignore_index=True)
print(len(postulaciones))
postulaciones2=0
gc.collect()
postulaciones.drop_duplicates(['idaviso','idpostulante'],keep='first',inplace=True)
print(len(postulaciones))

16511335
16502011


In [13]:
postulaciones=pd.merge(postulaciones,postulantes,on='idpostulante')
postulaciones=pd.merge(postulaciones,avisos,on='idaviso')
postulaciones=pd.merge(postulaciones,vistas,on=['idaviso','idpostulante'],how='left')
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad
3187319,1112315655,wVkGPRl,1,MASC,20.0,-,-,-,-,-,...,False,False,False,False,False,False,False,True,False,NaN
7477228,1111950343,2zPYLm0,0,MASC,22.0,-,-,Graduado,-,Graduado,...,False,False,False,False,False,False,False,False,False,NaN
1115960,1112286220,X95dwXd,0,FEM,33.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,False,NaN
11153842,1112294631,5mPQd9N,1,MASC,31.0,-,-,-,-,-,...,False,True,False,False,False,False,False,True,False,NaN
14280242,1112395177,kPjre91,0,MASC,21.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,False,NaN
12778303,1112198541,JBm0J6O,0,FEM,31.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,NaN
12735161,1112363492,EdRv4N,1,MASC,53.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.006250
11616662,1112318980,4rWojmA,1,MASC,28.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,NaN
14704077,1112386623,2zPdbNw,0,MASC,28.0,-,Graduado,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,NaN
2453400,1112299050,qe2V08L,0,MASC,30.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,NaN


In [14]:
postulaciones.cantidad=postulaciones.cantidad.fillna(0)
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad
13414964,1112461054,EBwJM6,0,FEM,27.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,True,False,True,0.000000
2311002,1112339374,6rZOpmv,1,MASC,38.0,-,-,-,-,Graduado,...,True,False,False,False,False,False,False,False,False,0.142857
14485246,1112479559,8MPJ3mW,0,NO_DECLARA,200.0,-,-,-,-,Graduado,...,False,False,False,False,False,True,False,False,False,0.000000
13572282,1112261344,YZDzZ,0,NO_DECLARA,200.0,-,-,-,-,-,...,False,False,False,False,False,False,False,True,False,0.000000
14633776,1112437855,96Xo269,0,MASC,18.0,-,-,-,-,-,...,False,True,False,False,False,False,False,False,False,0.000000
6953360,1112365422,Eze9D52,1,FEM,19.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,False,0.000000
10230865,1111922408,0zd0ZWr,0,FEM,39.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.000000
10098293,1111784393,BmDq6vE,0,MASC,50.0,-,-,-,En Curso,-,...,False,True,False,False,True,False,False,False,False,0.000000
2123680,1111620061,YjQJDlP,0,FEM,22.0,-,-,-,-,Graduado,...,False,False,True,False,False,True,False,False,False,0.000000
6556673,1112263226,ak5VJx9,1,MASC,23.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.000000


In [15]:
postulaciones.columns

Index(['idaviso', 'idpostulante', 'sepostulo', 'sexo', 'edad', 'Doctorado',
       'Master', 'Otro', 'Posgrado', 'Secundario', 'Terciario/Técnico',
       'Universitario', 'titulo', 'nombre_zona', 'tipo_de_trabajo',
       'nivel_laboral', 'nombre_area', 'denominacion_empresa',
       'titulo_uppercase', 'pide_hombre', 'pide_mujer', 'ingles',
       'experiencia', 'paquete_office', 'liderazgo', 'secundario', 'viajar',
       'empresa_importante', 'capacitacion', 'remuneracion_pretendida',
       'graduados', 'estudiantes', 'conocimientos_especificos',
       'buen_ambiente', 'discriminacion', 'cantidad'],
      dtype='object')

In [17]:
postulaciones.drop(['discriminacion','buen_ambiente'],axis=1,inplace=True)

In [18]:
train, test=train_test_split(postulaciones,test_size=0.10)
print("Train: ",len(train),"Test: ",len(test))
features=list(postulaciones.columns)
features.remove('idaviso')
features.remove('idpostulante')
features.remove('sepostulo')
features.remove('titulo')
features.remove('denominacion_empresa')

x_train=train[features]
y_train=train['sepostulo']

x_test=test[features]
y_test=test['sepostulo']
postulaciones=0
vistas2=0
train=0
test=0

Train:  14477569 Test:  1608619


In [19]:
gc.collect()

144

In [20]:
c=lgb.LGBMClassifier(learning_rate=0.01,objective='binary',num_leaves=7000,max_depth=18,n_estimators=10,colsample_bytree=0.9,n_jobs=-1,random_state=0,silent=False,subsample=1,sumsample_freq=1)

dt=c.fit(x_train,y_train,eval_set=[(x_test,y_test)],verbose=True)
#c = joblib.load("../../Data/fiuba_entrenamiento/gian/modelos/....pkl")

[1]	valid_0's binary_logloss: 0.689847
[2]	valid_0's binary_logloss: 0.686606
[3]	valid_0's binary_logloss: 0.683416
[4]	valid_0's binary_logloss: 0.679415
[5]	valid_0's binary_logloss: 0.67524
[6]	valid_0's binary_logloss: 0.671176
[7]	valid_0's binary_logloss: 0.667096
[8]	valid_0's binary_logloss: 0.663394
[9]	valid_0's binary_logloss: 0.659533
[10]	valid_0's binary_logloss: 0.65673


In [21]:
score=c.score(x_test,y_test)*100
print(score)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


80.70717802040134


In [22]:
joblib.dump(c, "../../Data/fiuba_entrenamiento/gian/modelos/liviano v3.pkl")

['../../Data/fiuba_entrenamiento/gian/modelos/liviano v3.pkl']

In [23]:
prediccion = pd.read_csv("../../Data/fiuba_entrenamiento/test_final_100k.csv")
print(len(prediccion))
prediccion=pd.merge(prediccion,postulantes,on='idpostulante',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,avisos,on='idaviso',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,vistas,on=['idaviso','idpostulante'],how='left')
prediccion.cantidad=prediccion.cantidad.fillna(0)
prediccion.sample(10)

100000
100000
100000


,id,idaviso,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad
68751,35988,1112411356,ZN1bjZ,FEM,31.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,False,0.000000
24003,74124,1112463662,YGq2mr,FEM,32.0,-,-,Graduado,-,Graduado,...,False,False,False,False,False,False,True,False,True,0.017857
76421,22660,1112293476,exGNAB,FEM,39.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.000000
97854,97656,1112502059,Z2qljb,FEM,34.0,-,-,-,-,-,...,False,True,False,False,False,False,False,False,True,0.000000
68813,44614,1112426827,RzrlMkx,FEM,28.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,True,False,0.000000
44931,41679,1112421818,5Q2xbq,FEM,32.0,-,-,En Curso,-,Graduado,...,False,False,True,False,False,False,False,True,False,0.000000
42965,1572,1110895236,5qzKbM,FEM,37.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.000000
66892,35080,1112409885,EX8lmJ,MASC,34.0,-,-,-,-,Graduado,...,False,True,False,False,False,False,False,False,False,0.000000
13502,60444,1112454993,Ezla5dJ,NO_DECLARA,200.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,False,0.032432
28503,28927,1112366913,kPjOjXJ,MASC,45.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,False,0.004484


In [26]:
x_final=prediccion[features]
y_final=c.predict_proba(x_final)
y_final

array([[0.52899383, 0.47100617],
       [0.52735136, 0.47264864],
       [0.52899383, 0.47100617],
       ...,
       [0.53868748, 0.46131252],
       [0.53964404, 0.46035596],
       [0.54034123, 0.45965877]])

In [27]:
y_final2=[]
print(c.classes_)
print(len(y_final))
for elem in y_final:
    y_final2.append(elem[1])
print(len(y_final2))

[0 1]
100000
100000


In [28]:
prediccion['sepostulo']=y_final2

In [29]:
prediccion.head()

,id,idaviso,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad,sepostulo
0,0,739260,6M9ZQR,FEM,42.0,Graduado,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,0.0,0.471006
1,1,739260,6v1xdL,MASC,30.0,-,-,Graduado,-,-,...,False,False,False,False,False,False,False,False,0.0,0.472649
2,2,739260,ezRKm9,FEM,36.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,0.0,0.471006
3,3,758580,1Q35ej,MASC,68.0,-,-,-,Graduado,Graduado,...,False,False,False,False,False,False,False,False,0.0,0.464068
4,4,758580,EAN4J6,FEM,32.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,0.0,0.455997


In [30]:
prediccion.drop(features,axis=1,inplace=True)
prediccion.drop(['idaviso','idpostulante','titulo','denominacion_empresa'],axis=1,inplace=True)
prediccion.head()

,id,buen_ambiente,discriminacion,sepostulo
0,0,False,False,0.471006
1,1,False,False,0.472649
2,2,False,False,0.471006
3,3,False,False,0.464068
4,4,False,False,0.455997


In [31]:
prediccion.drop(['discriminacion','buen_ambiente'],axis=1,inplace=True)

In [32]:
prediccion.to_csv("../../Data/fiuba_entrenamiento/gian/predicciones/liviano v3.csv",index=False)

In [33]:
prediccion.sepostulo.mean()

0.49879591172253596